In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
bat_data = pd.read_csv("Clean_Bat.csv",index_col = "Unnamed: 0")
bat_data.fillna(0,inplace=True)
pitch_data = pd.read_csv("Clean_Pitch.csv",index_col = "Unnamed: 0")
pitch_data.fillna(0,inplace=True)
prior_games = pd.read_csv("Previous_Games.csv",index_col="Unnamed: 0")

In [3]:
total_sum_df = pd.DataFrame()
rolling_window_df = pd.read_csv("7_Day_Window_Bat.csv",index_col = "Unnamed: 0")
start_date = rolling_window_df.iloc[-1]['Date']
x = 0
while x < len(prior_games[prior_games['Date'] > start_date].Date.unique()):
    team_date_df = pd.DataFrame()
    date_list = prior_games[prior_games['Date'] > start_date].Date.unique()[x:x+7]
    for date in date_list:
        date_df = bat_data[bat_data['Date'] == date]
        team_df = pd.DataFrame()
        for team in date_df.Team.unique():
            temp_df = pd.DataFrame(date_df.groupby("Team").get_group(team)[['PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']].sum()).T
            temp_df['Home_Away'] = int(date_df.Home_Away.iloc[0])
            temp_df['Date'] = date_df.Date.iloc[0]
            temp_df['Team'] = team
            temp_df.set_index("Team",inplace=True)
            team_df = pd.concat([team_df,temp_df])
        team_date_df = pd.concat([team_date_df,team_df])
    team_date_df.reset_index(inplace=True)
    for team in team_date_df.Team.unique():
        team_df = team_date_df
        team_df = team_df[team_df['Team'] == team]
        temp_df = pd.DataFrame(team_df.sum()[['PA','AB','R','H','2B','3B','HR','RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']]).T
        temp_df['Date'] = team_df.iloc[-1]['Date']
        temp_df['Team'] = team
        temp_df['Home_Away'] = team_df.iloc[-1].Home_Away
        total_sum_df = pd.concat([total_sum_df,temp_df])
    x+=1
if len(total_sum_df) > 0:
    df_2022 = total_sum_df[total_sum_df['Date'] < "2023-01-01"]
    post_first_week = total_sum_df[total_sum_df['Date'] > "2023-04-05"]
    final_df = pd.concat([df_2022,post_first_week])
    final_df.to_csv("7_Day_Window_Bat.csv")

In [4]:
total_sum_df = pd.DataFrame()
rolling_window_df = pd.read_csv("7_Day_Window_Pitch.csv",index_col = "Unnamed: 0")
start_date = rolling_window_df.iloc[-1]['Date']
x = 0
while x < len(prior_games[prior_games['Date'] > start_date].Date.unique()):
    team_date_df = pd.DataFrame()
    date_list = prior_games[prior_games['Date'] > start_date].Date.unique()[x:x+7]
    for date in date_list:
        date_df = bat_data[bat_data['Date'] == date]
        team_df = pd.DataFrame()
        for team in date_df.Team.unique():
            temp_df = pd.DataFrame(date_df.groupby("Team").get_group(team)[['GS', 'IP', 'H', 'R', 'ER','BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB','CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']].sum()).T
            temp_df['Home_Away'] = int(date_df.Home_Away.iloc[0])
            temp_df['Date'] = date_df.Date.iloc[0]
            temp_df['Team'] = team
            temp_df.set_index("Team",inplace=True)
            team_df = pd.concat([team_df,temp_df])
        team_date_df = pd.concat([team_date_df,team_df])
    team_date_df.reset_index(inplace=True)
    for team in team_date_df.Team.unique():
        team_df = team_date_df
        team_df = team_df[team_df['Team'] == team]
        temp_df = pd.DataFrame(team_df.sum()[['GS', 'IP', 'H', 'R', 'ER','BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB','CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']]).T
        temp_df['Date'] = team_df.iloc[-1]['Date']
        temp_df['Team'] = team
        temp_df['Home_Away'] = team_df.iloc[-1].Home_Away
        total_sum_df = pd.concat([total_sum_df,temp_df])
    x+=1
if len(total_sum_df) > 0:
    df_2022 = total_sum_df[total_sum_df['Date'] < "2023-01-01"]
    post_first_week = total_sum_df[total_sum_df['Date'] > "2023-04-05"]
    final_df = pd.concat([df_2022,post_first_week])
    final_df.drop(columns = ['GB/FB','LD','PU'],inplace=True)
    final_df.to_csv("7_Day_Window_Pitch.csv")